In [1]:
from keras.layers import Reshape, InputLayer, Input, MaxPooling2D, Conv2D, Dense, Flatten, Dropout
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import os

2024-05-26 20:31:49.328567: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 20:31:49.328791: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-26 20:31:49.330846: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-26 20:31:49.354494: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 20:31:49.736934: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
# Check tensorflow version
tf.__version__

'2.16.1'

In [3]:
label_file_path = '../data/train/outputs/image_descriptions.txt'

In [4]:
# Define a directory for saved files
output_dir = '../data/train/snippets/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [5]:
# Constants
img_shape_full = (150, 150, 1)
num_classes = 2

In [6]:
# Get a list of all .npy files in the directory
npy_files = [f for f in os.listdir(output_dir) if f.endswith('.npy')]

In [7]:
# Read labels from the text file
labels = {}
with open(label_file_path, 'r') as file:
    for line in file:
        filename, label = line.strip().split(',')
        labels[filename] = int(label)

# Load images and their labels based on the filenames
images = []
image_labels = []
for npy_file in sorted(os.listdir(output_dir)): # SUS
    if npy_file.endswith('.npy') and npy_file in labels:
        img_array = np.load(os.path.join(output_dir, npy_file))
        images.append(img_array)
        image_labels.append(labels[npy_file])


In [8]:
print("Number of images:", len(images))
print("Number of labels:", len(image_labels))
assert len(images) == len(image_labels)

print("Image shape:", images[0].shape)

Number of images: 29
Number of labels: 29
Image shape: (150, 150)


In [9]:
# Assuming all images are the same size and reshaped properly for input to a CNN
images = np.array(images).reshape(-1, 150, 150, 1)  # Reshape for CNN, change shape as necessary

# Start constructing the Keras Sequential model.
model = Sequential()

# Add an input layer which is similar to a feed_dict in TensorFlow.
model.add(InputLayer(input_shape=img_shape_full))

# First convolutional layer with ReLU-activation and max-pooling.
model.add(Conv2D(kernel_size=3, strides=1, filters=1, padding='same',
                activation='relu', name='layer_conv1'))
# model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2, strides=2))


model.add(Conv2D(kernel_size=3, strides=1, filters=1, padding='same',
                activation='relu', name='layer_conv2'))
# model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2, strides=2))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Last fully-connected layer with softmax-activation for use in classification.
model.add(Dense(num_classes, activation='softmax'))

# Compile the model (assuming this is for a classification task)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Show a summary of the model's architecture
model.summary()

/home/ahamboeck/venv/keras/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2024-05-26 20:31:50.651807: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-26 20:31:50.652101: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer_conv1 (Conv2D)            │ (None, 150, 150, 1)    │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_conv2 (Conv2D)            │ (None, 75, 75, 1)      │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1369)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,740 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,760 (10.78 KB)

 Trainable params: 2,760 (10.78 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
base_path = '../models/model_no_dropout_2_conv/'
path = base_path + 'model_no_dropout_2_conv'

In [11]:
# Assuming 'model' is your Keras model

# Open a file in write mode
with open(path + '.txt', 'w') as f:
    # Pass the file handle to the print function of model.summary()
    model.summary(print_fn=lambda x: f.write(x + '\n'))

print("Model summary saved to %s", path + ".txt")


Model summary saved to %s ../models/model_no_dropout_2_conv/model_no_dropout_2_conv.txt


In [12]:
# Convert lists to numpy arrays
image_labels = np.array(image_labels)
print(image_labels)

# Convert labels to one-hot encoding
image_labels = to_categorical(image_labels)
print(image_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, image_labels, test_size=0.3, random_state=42)

# Standardize pixel values to mean 0 and standard deviation 1
mean = np.mean(X_train, axis=(0, 1, 2), keepdims=True)
std = np.std(X_train, axis=(0, 1, 2), keepdims=True)

# Save to a file
np.save(base_path + 'mean.npy', mean)
np.save(base_path + 'std.npy', std)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

[0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [13]:
# Fit the model
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=10,
    epochs=7,
    verbose=1,
    validation_data=(X_test, y_test)  # Use test data for validation
)

model.save(path + '.keras')

Epoch 1/7
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.2667 - loss: 0.8085 - val_accuracy: 0.3333 - val_loss: 0.7107
Epoch 2/7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3000 - loss: 0.7406 - val_accuracy: 0.5556 - val_loss: 0.6868
Epoch 3/7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5667 - loss: 0.7060 - val_accuracy: 0.5556 - val_loss: 0.6645
Epoch 4/7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6667 - loss: 0.6769 - val_accuracy: 0.7778 - val_loss: 0.6431
Epoch 5/7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8333 - loss: 0.6349 - val_accuracy: 1.0000 - val_loss: 0.6213
Epoch 6/7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8000 - loss: 0.6146 - val_accuracy: 1.0000 - val_loss: 0.5995
Epoch 7/7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7667 - loss: 0.5869 - val_accuracy: 1.0000 - val_loss: 0.5771


In [14]:
# Plot training & validation accuracy values and save the plot
plt.figure(figsize=(6, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy - No Dropout - Single Convolution')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(path + '_accuracy.png')
plt.close()  # Close the plot to free up memory

# Plot training & validation loss values and save the plot
plt.figure(figsize=(6, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss - No Dropout - Single Convolution')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(path + '_loss.png')
plt.close()  # Close the plot to free up memory

# Predict the values from the test dataset
Y_pred = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1) 
Y_true = np.argmax(y_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(Y_true, Y_pred_classes)

# Plot confusion matrix and save it
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
plt.title('Confusion Matrix - No Dropout - Single Convolution')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig(path + '_confusion_matrix.png')
plt.close()  # Close the plot to free up memory

print("Plots saved in the 'models' directory.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Plots saved in the 'models' directory.


In [33]:
# Load models
model1 = load_model('../models/model_dropout/model_dropout.keras')
model2 = load_model('../models/model_no_dropout/model_no_dropout.keras')

In [34]:
# Assuming X_test and y_test are your test datasets
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [35]:
y_pred1_classes = np.argmax(y_pred1, axis=1)
y_pred2_classes = np.argmax(y_pred2, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [36]:
cm1 = confusion_matrix(y_true_classes, y_pred1_classes)
cm2 = confusion_matrix(y_true_classes, y_pred2_classes)

In [37]:
confusion_matrix_path = '../models/confusion_matrices/'

In [38]:
pd.DataFrame(cm1).to_csv(confusion_matrix_path + 'model_dropout_1_conv.csv', index=False)
pd.DataFrame(cm2).to_csv(confusion_matrix_path + 'model_no_dropout_1_conv.csv', index=False)

In [39]:
# Load the confusion matrices
cm1 = pd.read_csv(confusion_matrix_path + 'model_dropout_1_conv.csv').to_numpy()
cm2 = pd.read_csv(confusion_matrix_path + 'model_no_dropout_1_conv.csv').to_numpy()

In [40]:
# b and c are the off-diagonal elements in McNemar's 2x2 table
b = cm1[0, 1]
c = cm1[1, 0]
contingency_table = np.array([[0, b],
                              [c, 0]])

In [41]:
mcnemar_path = '../models/mcnemar_test_results/'

In [43]:
# Perform the McNemar test, assuming both b and c are sufficiently large to avoid the exact method
result = mcnemar(contingency_table, exact=False)
print("McNemar's Test Statistic:", result.statistic)
print("P-value:", result.pvalue)

# Save the p-value to a file
with open(mcnemar_path + 'mcnemar_1_conv.txt', 'w') as file:
    file.write(f"McNemar's Test Statistic: {result.statistic}\n")
    file.write(f"P-value: {result.pvalue}")


McNemar's Test Statistic: inf
P-value: 0.0
